In [18]:
import os
os.makedirs("../docs", exist_ok=True)

# EMBUTIR o plotly.js para não depender de CDN
fig_candle.write_html("../docs/candle.html", include_plotlyjs="inline", full_html=True)
fig_vol.write_html("../docs/volume.html", include_plotlyjs="inline", full_html=True)
fig_sma.write_html("../docs/sma20.html", include_plotlyjs="inline", full_html=True)

# Opcional: garantir que o Pages não rode Jekyll (não é estritamente necessário, mas ajuda)
open("../docs/.nojekyll", "w").close()

print("HTMLs regravados no /docs com Plotly embutido.")


HTMLs regravados no /docs com Plotly embutido.


In [19]:
import pandas as pd, plotly.graph_objects as go, plotly.io as pio, os
pio.renderers.default = "vscode"

# 1) Ler o CSV limpo e garantir tipos
df = pd.read_csv("../data/aapl.csv", parse_dates=[0], index_col=0).sort_index()
df = df[["Open","High","Low","Close","Volume"]].apply(pd.to_numeric, errors="coerce")

# 2) Remover linhas problemáticas (NaN ou OHLC inconsistentes)
df = df.dropna()
df = df[(df["High"]>=df[["Open","Close"]].max(axis=1)) &
        (df["Low"] <=df[["Open","Close"]].min(axis=1))]

# 3) Construir o Candlestick com arrays "puros"
x      = df.index.strftime("%Y-%m-%d").to_numpy()
open_  = df["Open"].astype(float).to_numpy()
high   = df["High"].astype(float).to_numpy()
low    = df["Low"].astype(float).to_numpy()
close  = df["Close"].astype(float).to_numpy()

fig_candle = go.Figure(data=[go.Candlestick(x=x, open=open_, high=high, low=low, close=close)])
fig_candle.update_layout(title="Candlestick", xaxis_rangeslider_visible=False,
                         width=1100, height=600, margin=dict(l=40,r=20,t=60,b=40))

# 4) Salvar página auto-contida (Plotly embutido)
os.makedirs("../docs", exist_ok=True)
from plotly.io import to_html
html = f"""<!doctype html><html lang="pt-br"><head>
<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>Candlestick</title>
<style>body{{font-family:system-ui,Segoe UI,Roboto,Arial;margin:0}}
.wrapper{{max-width:1200px;margin:2rem auto;padding:1rem}}</style>
</head><body><div class="wrapper"><h1>Candlestick</h1>
{to_html(fig_candle, full_html=False, include_plotlyjs="inline")}
<p><a href="./">← Voltar</a></p></div></body></html>"""
with open("../docs/candle.html","w",encoding="utf-8") as f: f.write(html)

# (garante que o GitHub Pages não processe nada)
open("../docs/.nojekyll", "w").close()

print("docs/candle.html recriado.")


docs/candle.html recriado.


In [20]:
import pandas as pd, numpy as np, plotly.graph_objects as go, os, plotly.io as pio
from plotly.io import to_html
pio.renderers.default = "vscode"

# === carregar base limpa ===
df = pd.read_csv("../data/aapl.csv", parse_dates=[0], index_col=0).sort_index()
df = df[["Open","High","Low","Close","Volume"]].apply(pd.to_numeric, errors="coerce").dropna()

# === RSI(14) ===
window = 14
delta = df["Close"].diff()
gain  = delta.clip(lower=0).rolling(window).mean()
loss  = (-delta.clip(upper=0)).rolling(window).mean()
rs    = gain / (loss.replace(0, np.nan))
df["RSI"] = 100 - (100 / (1 + rs))

fig_rsi = go.Figure(go.Scatter(x=df.index, y=df["RSI"], mode="lines", name="RSI(14)"))
fig_rsi.add_hrect(y0=30, y1=70, fillcolor="lightgray", opacity=0.2, line_width=0)
fig_rsi.update_layout(title="RSI(14)", yaxis_title="Índice", width=1100, height=400)

# === MACD (12,26,9) ===
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26
df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
df["Hist"] = df["MACD"] - df["Signal"]

fig_macd = go.Figure()
fig_macd.add_scatter(x=df.index, y=df["MACD"],   mode="lines", name="MACD")
fig_macd.add_scatter(x=df.index, y=df["Signal"], mode="lines", name="Signal")
fig_macd.add_bar(x=df.index, y=df["Hist"], name="Hist")
fig_macd.update_layout(title="MACD (12,26,9)", width=1100, height=450)

# === Retorno acumulado ===
ret = df["Close"].pct_change().fillna(0.0)
df["CumRet"] = (1 + ret).cumprod() - 1
fig_ret = go.Figure(go.Scatter(x=df.index, y=df["CumRet"], mode="lines", name="Retorno acumulado"))
fig_ret.update_layout(title="Retorno acumulado (Close)", yaxis_tickformat=".0%", width=1100, height=450)

# === salvar HTMLs no /docs ===
def save_page(fig, path, title):
    fig.update_layout(margin=dict(l=40,r=20,t=60,b=40))
    html = f"""<!doctype html><html lang="pt-br"><head>
<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>{title}</title></head><body style="font-family:system-ui,Segoe UI,Roboto,Arial;margin:0">
<div style="max-width:1200px;margin:2rem auto;padding:1rem">
<h1>{title}</h1>
{to_html(fig, full_html=False, include_plotlyjs="inline")}
<p><a href="./">← Voltar</a></p></div></body></html>"""
    with open(path, "w", encoding="utf-8") as f: f.write(html)

os.makedirs("../docs", exist_ok=True)
save_page(fig_rsi,  "../docs/rsi.html",  "RSI(14)")
save_page(fig_macd, "../docs/macd.html", "MACD (12,26,9)")
save_page(fig_ret,  "../docs/retorno.html", "Retorno acumulado")
open("../docs/.nojekyll","w").close()

print("RSI/MACD/Retorno salvos no /docs")


RSI/MACD/Retorno salvos no /docs


In [1]:
# -*- coding: utf-8 -*-
import os
from datetime import datetime
from typing import Optional

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.io import to_html
from pandas_datareader import data as web
import yfinance as yf

# ----------------------------
# CONFIG
# ----------------------------
TICKERS = ["AAPL", "MSFT"]
START, END = datetime(2024, 1, 1), datetime(2024, 8, 31)
DOCS_DIR = "../docs"
PLOTLY_CDN = "https://cdn.plot.ly/plotly-2.35.2.min.js"

WIDTH, HEIGHT = 1100, 560
MARGIN = dict(l=40, r=20, t=60, b=40)
AXGRID = dict(showgrid=True, gridcolor="#334155", zeroline=False)

os.makedirs(DOCS_DIR, exist_ok=True)

# ----------------------------
# HELPERS (dados)
# ----------------------------
NEEDED = ["open", "high", "low", "close", "volume"]

def _normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]
    aliases = {
        "adj close": "close",
        "close*": "close",
        "vol": "volume",
        "turnover": "volume",
    }
    df = df.rename(columns={k: v for k, v in aliases.items() if k in df.columns})
    return df

def _order_and_check(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    if df is None or df.empty:
        return None
    df = _normalize_cols(df).sort_index()
    missing = [c for c in NEEDED if c not in df.columns]
    if missing:
        return None
    df = df[NEEDED].apply(pd.to_numeric, errors="coerce").dropna()
    # sanidade OHLC
    df = df[
        (df["high"] >= df[["open", "close"]].max(axis=1))
        & (df["low"] <= df[["open", "close"]].min(axis=1))
    ]
    return df if not df.empty else None

def fetch_stooq(ticker: str) -> Optional[pd.DataFrame]:
    try:
        df = web.DataReader(ticker, "stooq", START, END)
        return _order_and_check(df)
    except Exception:
        return None

def fetch_yf(ticker: str) -> Optional[pd.DataFrame]:
    try:
        df = yf.download(ticker, start=START, end=END, auto_adjust=False, progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        return _order_and_check(df)
    except Exception:
        return None

def fetch_any(ticker: str) -> Optional[pd.DataFrame]:
    df = fetch_stooq(ticker)
    if df is not None:
        print(f"✓ {ticker}: stooq")
        return df
    df = fetch_yf(ticker)
    if df is not None:
        print(f"✓ {ticker}: yfinance")
        return df
    print(f"⛔ {ticker}: sem dados válidos.")
    return None

# ----------------------------
# HELPERS (figuras)
# ----------------------------
def stylize(fig: go.Figure, title: Optional[str] = None) -> go.Figure:
    fig.update_layout(
        template="plotly_dark",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="#e5e7eb"),
        width=WIDTH,
        height=HEIGHT,
        margin=MARGIN,
        title=title or fig.layout.title.text,
    )
    fig.update_xaxes(**AXGRID)
    fig.update_yaxes(**AXGRID)
    return fig

def fig_candle(df, title):
    f = go.Figure(
        go.Candlestick(
            x=df.index, open=df["open"], high=df["high"], low=df["low"], close=df["close"]
        )
    )
    f.update_layout(xaxis_rangeslider_visible=False)
    return stylize(f, title)

def fig_volume(df):
    f = go.Figure(go.Bar(x=df.index, y=df["volume"], name="Volume"))
    f.update_layout(yaxis_title="Shares")
    return stylize(f, "Volume")

def fig_sma(df, window=20):
    d = df.copy()
    d["sma"] = d["close"].rolling(window, min_periods=1).mean()
    f = go.Figure()
    f.add_scatter(x=d.index, y=d["close"], name="Close", mode="lines")
    f.add_scatter(x=d.index, y=d["sma"], name=f"SMA {window}", mode="lines")
    f.update_layout(yaxis_title="Preço")
    return stylize(f, f"Média Móvel {window}d")

def fig_rsi(df, window=14):
    delta = df["close"].diff()
    gain = delta.clip(lower=0).rolling(window).mean()
    loss = (-delta.clip(upper=0)).rolling(window).mean().replace(0, np.nan)
    rsi = 100 - (100 / (1 + (gain / loss)))
    f = go.Figure(go.Scatter(x=df.index, y=rsi, name=f"RSI({window})", mode="lines"))
    f.add_hrect(y0=30, y1=70, fillcolor="lightgray", opacity=0.2, line_width=0)
    return stylize(f, f"RSI({window})")

def fig_macd(df):
    ema12 = df["close"].ewm(span=12, adjust=False).mean()
    ema26 = df["close"].ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    hist = macd - signal
    f = go.Figure()
    f.add_scatter(x=df.index, y=macd, name="MACD", mode="lines")
    f.add_scatter(x=df.index, y=signal, name="Signal", mode="lines")
    f.add_bar(x=df.index, y=hist, name="Hist")
    return stylize(f, "MACD (12,26,9)")

def fig_bollinger(df, window=20, k=2):
    ma = df["close"].rolling(window).mean()
    std = df["close"].rolling(window).std()
    upper, lower = ma + k * std, ma - k * std
    f = go.Figure()
    f.add_scatter(x=df.index, y=df["close"], name="Close", mode="lines")
    f.add_scatter(x=df.index, y=upper, name=f"BBand+{k}σ", mode="lines")
    f.add_scatter(x=df.index, y=ma, name=f"SMA {window}", mode="lines")
    f.add_scatter(x=df.index, y=lower, name=f"BBand-{k}σ", mode="lines")
    return stylize(f, f"Bollinger Bands ({window},{k}σ)")

def fig_atr(df, window=14):
    prev_close = df["close"].shift(1)
    tr = pd.concat(
        [
            (df["high"] - df["low"]),
            (df["high"] - prev_close).abs(),
            (df["low"] - prev_close).abs(),
        ],
        axis=1,
    ).max(axis=1)
    atr = tr.rolling(window).mean()
    f = go.Figure(go.Scatter(x=df.index, y=atr, mode="lines", name="ATR"))
    return stylize(f, f"ATR({window})")

def fig_ret_acum(df):
    ret = df["close"].pct_change().fillna(0)
    cum = (1 + ret).cumprod() - 1
    f = go.Figure(go.Scatter(x=df.index, y=cum, mode="lines", name="Ret Acum"))
    f.update_layout(yaxis_tickformat=".0%")
    return stylize(f, "Retorno acumulado")

def fig_heatmap_monthly(df):
    # 'ME' = month end (evita o warning do 'M' deprecado)
    mret = df["close"].resample("ME").last().pct_change().dropna()
    table = mret.to_frame("ret")
    table["Ano"] = table.index.year
    table["Mês"] = table.index.strftime("%b")
    pivot = table.pivot(index="Ano", columns="Mês", values="ret").fillna(0)
    order = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    pivot = pivot.reindex(columns=[m for m in order if m in pivot.columns])

    import plotly.express as px
    f = px.imshow(
        pivot * 100,
        text_auto=".1f",
        aspect="auto",
        labels=dict(color="%"),
        color_continuous_scale="RdYlGn",
    )
    f.update_layout(coloraxis_colorbar=dict(ticksuffix="%"))
    return stylize(f, "Retornos mensais (%)")

# ----------------------------
# HTML (um painel por ticker, gráficos empilhados)
# ----------------------------
PAGE_CSS = """
:root{
  --bg:#0b1220; --card:#0f172a; --text:#e5e7eb; --muted:#9ca3af;
  --accent:#93c5fd; --border:#1f2937
}
*{box-sizing:border-box}
body{margin:0;background:var(--bg);color:var(--text);font-family:system-ui,Segoe UI,Roboto,Arial}
.wrapper{max-width:1250px;margin:2rem auto;padding:1rem}
.header{display:flex;align-items:center;justify-content:space-between;gap:.75rem}
.muted{color:var(--muted)}
h1{font-size:clamp(28px,3.0vw,42px);margin:0 0 .25rem}
h2{font-size:clamp(20px,2.3vw,28px);margin:.75rem 0 .5rem}
.card{background:var(--card);border:1px solid var(--border);border-radius:14px;padding:1rem;margin-bottom:18px;box-shadow:0 1px 3px rgba(0,0,0,.35)}
a, a:visited{color:#cbd5e1}
footer{margin-top:1.25rem}
"""

def write_panel_page(ticker: str, df: pd.DataFrame):
    title = f"{ticker} — Painel"
    sections = [
        ("Candlestick", fig_candle(df, f"{ticker} — Candlestick")),
        ("Volume",      fig_volume(df)),
        ("Média Móvel 20d", fig_sma(df, 20)),
        ("RSI(14)",     fig_rsi(df, 14)),
        ("MACD",        fig_macd(df)),
        ("Bollinger",   fig_bollinger(df, 20, 2)),
        ("ATR(14)",     fig_atr(df, 14)),
        ("Retorno acumulado", fig_ret_acum(df)),
        ("Heatmap mensal",    fig_heatmap_monthly(df)),
    ]

    # Monta todas as seções de uma vez (empilhadas)
    body_blocks = []
    for heading, fig in sections:
        block = f"""
        <section class="card">
          <h2>{heading}</h2>
          {to_html(fig, full_html=False, include_plotlyjs=False, default_width="100%")}
          <p class="muted">Fontes: Stooq / Yahoo / yfinance / CSV</p>
        </section>
        """
        body_blocks.append(block)

    html = f"""<!doctype html>
<html lang="pt-br">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>{title}</title>
  <script src="{PLOTLY_CDN}"></script>
  <style>{PAGE_CSS}</style>
</head>
<body>
  <div class="wrapper">
    <div class="header">
      <div>
        <h1>{title}</h1>
        <div class="muted">Plotly • múltiplas fontes • Dark</div>
      </div>
      <div><a href="./">← Home</a></div>
    </div>
    {''.join(body_blocks)}
    <footer class="muted">Feito por Pasqual — ciência com método, arte com medida.</footer>
  </div>
</body>
</html>"""

    out_path = os.path.join(DOCS_DIR, f"{ticker}.html")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(html)

def write_home(cards):
    home = f"""<!doctype html>
<html lang="pt-br">
<head>
<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>Observatório Financeiro</title>
<style>
{PAGE_CSS}
.grid{{display:grid;gap:1rem}}
@media(min-width:900px){{.grid{{grid-template-columns:repeat(2,1fr)}}}}
.card a{{text-decoration:none;color:inherit}}
.card a:hover{{text-decoration:underline}}
</style>
</head>
<body>
<div class="wrapper">
  <h1>Observatório Financeiro</h1>
  <p class="muted">Plotly • Stooq/Yahoo/yfinance • GitHub Pages</p>
  <div class="grid">
    {''.join([f'<div class="card"><h2><a href="{link}">{tk}</a></h2><div class="muted">Painel completo</div></div>' for tk, link in cards])}
  </div>
  <footer class="muted">Feito por Pasqual — ciência com método, arte com medida.</footer>
</div>
</body>
</html>"""
    with open(os.path.join(DOCS_DIR, "index.html"), "w", encoding="utf-8") as f:
        f.write(home)
    # garante que o GitHub Pages não rode Jekyll
    open(os.path.join(DOCS_DIR, ".nojekyll"), "w").close()

# ----------------------------
# EXECUÇÃO
# ----------------------------
cards = []
for tk in TICKERS:
    df = fetch_any(tk)
    if df is None:
        continue
    write_panel_page(tk, df)
    cards.append((tk, f"{tk}.html"))

write_home(cards)
print("✅ Gerado: /docs/index.html e /docs/<TICKER>.html com gráficos empilhados (dark).")


✓ AAPL: stooq
✓ MSFT: stooq
✅ Gerado: /docs/index.html e /docs/<TICKER>.html com gráficos empilhados (dark).
